In [ ]:
import pymongo
import datetime
import collections
from numpy import nan as NA
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import folium
import numpy as np
import pandas as pd
import scipy.stats

**** Reading in the first five rows of data/HibernationSurveyObservationsCleaned.csv using the head() command
to view the type of data and to get a feel for the dataset.I had initially used the uncleaned version so some of my 
work in the cells below was based on cleaning. Please ignore these cells. ****

In [ ]:
#Reading in the first five rows of data using the head() command to view the type of data and to get a feel for the dataset.
#hib_df = pd.read_csv('data/HibernationSurveyObservations.csv')
#hib_df.head()
hib_df = pd.read_csv('data/HibernationSurveyObservationsCleaned.csv')
hib_df.head()

In [ ]:
len(hib_df)

**** Checking the length of the dataframe and comparing it to the counts available from the NBMP website where the data was
obtained, I have made the assumption that each row acts as a single count, ie. a single bat.****

In [ ]:
len(hib_df['recordKey'])

In [ ]:
hib_df.columns

**** The columns in the dataframe provide a feel for the types of data ****

In [ ]:
#checking the names to find columns of interest to use unique.
hib_df.columns

****Checking data types****

In [ ]:
hib_df.dtypes

****The recordKey seems to correspond to numbers of bats because it has the same length as the dataframe, so looking 
at it more closely****

In [ ]:
hib_df.recordKey.describe()
#An overview of the numerical data.

In [ ]:
hib_df.featureKey.describe()

****It would seem that each row of the dataframe represents one bat count, because teh numerical values above are the same 
for featureKey and recordKey****

In [ ]:
#beginning converting the grid references to latitude and longitude to plot on a map
refs = []
for v in hib_df['gridReference']:
    if v not in refs:
        refs.append(v)
print(refs)

****The purpose of this was to convert the values so that they could be plotted on a map. I converted them using the notebook 
grid-refs-to-lat-lon.ipynb and then copied them into a spreadsheet data/latlong.csv****

In [ ]:
%run 'grid-refs-to-lat-lon.ipynb'
to_osgb36(refs)
#Using the grid-refs-to-lat-lon.ipynb notebook to convert the grid refs to latitude and longitude.

**** The count values for the observationID, recordKey, featureKey are consistent with the rows in the table
which implies that as described earlier, the records or rows are the count of the bats.****

In [ ]:
#This is not workable and will only be performed on the columns of interest
unique_values = pd.unique(hib_df.values.ravel())
len(unique_values)

In [ ]:
pd.unique(hib_df['commonName'])

****The bat names are consistent with the report provided - NBMP_Summary_2013_web.pdf****

In [ ]:
#groupeddata = hib_df.groupby(['organisationName'])
groupeddata = hib_df.groupby(['commonName'])
grouped = groupeddata['commonName'].aggregate('count')
groupeddata.size()
#experimenting with grouping to try to get a sutable format

****This plot looks good for the amount of bats over the whole period covered by the data****

In [ ]:
totals = groupeddata.sum()
grouped.plot(kind='bar')

In [ ]:
#Checking to see if startDate and endDate are to be converted to date format
hib_df.dtypes

****The dates are in object format so need to be converted****

In [ ]:
#Checking how far through 2014 the data goes
hib_df[hib_df['startDate'].str.contains('2014')]

****The data for 2014 only goes as far as February so this cannot be analysed in comparison with earlier years
However it explains why there is a disparity in the listed numbers on the NBN Gateway site and my analysis.****

In [ ]:
#They are object format so need to be converted
hib_df['endDate'] = pd.to_datetime(hib_df['endDate'])

In [ ]:
#They are object format so need to be converted
hib_df['startDate'] = pd.to_datetime(hib_df['startDate'])

In [ ]:
#Retrieving the minimum year
startYear = hib_df['startDate'].dt.year
minYear = startYear.min()
minYear

****The earliest date in the dataset is 1964****

In [ ]:
#Retrieving the maximum year
endYear = hib_df['endDate'].dt.year
maxYear = endYear.max()
maxYear

****The latest date in the dataset is 2014 (February)****

In [ ]:
#Checking that startDate and endDate are equal(they are so the column will now be date)
daterange = hib_df['endDate'] - hib_df['startDate']
daterange.unique()

Getting a count of the bat types below

In [ ]:
bat_df = hib_df[['commonName','startDate']]
pivoted = bat_df.pivot_table(index=['commonName'], aggfunc='count')
pivoted.reset_index('startDate',inplace=True)
pivoted = pivoted.rename(columns = {'startDate' : 'count'})
pivoted.head()

****Graphing bat counts over time to get a picture of general trends****

In [ ]:
hib_df.commonName.value_counts()

In [ ]:
new_df = hib_df.copy()
#Making a copy of the dataframe to keep the previous analysis intact.

****Making a copy of the dataframe to keep the previous analysis intact, adding the Counts column and extracting year.
This will be the format which will provide most of the rest of the analysis, based on the information it provides 
on the counts of bats over time.****

In [ ]:
new_df = new_df[['commonName','startDate']]
new_df['Count'] = new_df['startDate'].map(new_df.groupby(['startDate'])['commonName'].unique().apply(len))
new_df = new_df.rename(columns={'startDate': 'years'})
new_df['years'] = new_df.years.dt.year#changed to years
#created a new dataframe with selected columns in order to show the relevant data to see the bat counts over time.
new_df.head()

In [ ]:
new_year = new_df.groupby(['years'])

In [ ]:
new_year_totals = new_year.sum()

****The cells above and below show the total bat count over the time period****

In [ ]:
new_year_totals.plot()

In [ ]:
new_year_types = new_df.groupby(['years','commonName'])['Count'].sum()

In [ ]:
new_year_types.unstack()

****Total bat counts over time****

In [ ]:
plot_new_year_types = new_year_types.unstack().plot(figsize=(10,8))
plt.title('Bat Population of the UK over time, by decade', fontsize=20, color='Blue')
plt.xlabel('Years by decade')
plt.ylabel('Total Count')

****I have plotted similar information below, but the above looked tidier. I have left mine in though, because I might use 
some of it later****

In [ ]:
#Looking at the population of the common name Lesser Noctule over time because this was a small value on the total count.
bat = new_df[new_df['commonName']=='Lesser Noctule']
grp = bat.groupby('years').apply(lambda x: x['commonName'].value_counts()).unstack().fillna(0)
grp.head()

****Looking at the population of the common name Lesser Noctule over time because this was a small value on the total count****

In [ ]:
#Grouping as a dataframe
b = bat.groupby(['years', 'commonName'])[['Count']].sum()
bs = b.reset_index()
bs.head()

****Grouping and plotting Lesser Noctule to get a clearer view of populations****

In [ ]:
bs.plot(x='years', kind='bar', figsize=(8,6))
plt.title('Common Name "Lesser Noctule"', fontsize=15, color='Blue')
plt.xlabel('Years')
plt.ylabel('Total Count')
#Lesser Noctule (commonName) numbers over time.

In [ ]:
#Using pivot to reorder to plot without specifying the axis
bats = bs.pivot(index='years', columns='commonName').fillna(0)
bats.head()

In [ ]:
bats.plot(kind='bar', figsize=(10,10))
plt.title('Common Name "Lesser Noctule"', fontsize=20, color='Red')
plt.xlabel('Years')
plt.ylabel('Total Count')
#bat.plot(x=['years'], y=['Count'])

****Looks the same with both methods, no advantage to either***

In [ ]:
#Looking at the population of the common name Natterer's Bat over time.
natbat = new_df[new_df['commonName']=='Natterer\'s Bat']
#g = natbat.groupby('years').apply(lambda x: x['commonName'].value_counts()).unstack().fillna(0)
#g.head()

****Natterers bat is on the other end of the scale with high values, so plotting this one now****

In [ ]:
#Grouping by years and commonName
nb = natbat.groupby(['years', 'commonName'])[['Count']].sum()
bn = nb.reset_index()
bn.head()

In [ ]:
#Pivot to reorder as before
batnat = bn.pivot(index='years', columns='commonName').fillna(0)
batnat.head()

In [ ]:
#Plotting pivot because no need to specify axis
batnat.plot(kind='bar', figsize=(10,8))
plt.title('Common Name "Natterer\'s Bat"', fontsize=10, color='Blue')
plt.xlabel('Years')
plt.ylabel('Total Count')

****The large increases in later years look to be due to better collection and recording, however it looks like populations are
improving again after a dip, although because 2014 is incomplete this is ignored****

In [ ]:
cross = new_df.copy

****This shows the same trends as above****

In [ ]:
stuff = pd.crosstab(battiest['years'], battiest['Count'])
crosstab_long = stuff.stack().reset_index()
plt.scatter(crosstab_long['years'], 
            crosstab_long['Count'],
            s=np.sqrt(crosstab_long[0])*1.5
            )
plt.show()

In [ ]:
# The crosstab has reshaped the DataFrame - there is now a row for each unique value in the commonNmae column 
#of the original table, and a column for each unique value in the years column of the original table.
#At the intersection of each row and column there is the count of the number of times that row value and that column 
#value occur in the original table's rows.
# The result is a table bats_df.
bats_df = pd.crosstab(new_df['commonName'], new_df['years'], margins=True)
#bats_df.reset_index()
bats_df.head()

****This format allows me to plot the whole dataset more easily and also to set the years by decade to produce a chart, which 
otherwise is quite overloaded. In the EMA I intend to separate the decades to plot where appropriate.****

In [ ]:
pivoted = new_df.pivot_table(index=['commonName', 'years', 'Count'], aggfunc=np.sum)
pivoted.columns

****Convering the years to decades to plot****

In [ ]:
#Adding functions to select years
updated_bats = new_df.copy()
#if pnl[company].tail(1)['Active'].any()==1:
    #print 'yay'
def select_decade(y):
    for y in updated_bats.years:
        if y.all() > 1960:
            return years
def floor_decade(y):
    for y in updated_bats.years:
        return (y // 10) * 10

****Plotting decades****

In [ ]:
updated_bats['years'] = (updated_bats['years']// 10) * 10
grouped = updated_bats.groupby(['years'])
batter_df = grouped['commonName', 'Count'].sum()
batter_df.plot.bar()

****Grouping the dataframe by decade to tidy the data****

In [ ]:
battier = updated_bats.groupby(['years', 'commonName'])[['Count']].sum()
#battier.replace(np.nan, 0)
battier

In [ ]:
battiest = battier.reset_index()
battiest.head()

#flattening the index to plot

****Grouping by years and producing counts of name columns****

In [ ]:
counts = battiest.groupby('years').apply(lambda x: x['commonName'].value_counts()).unstack().fillna(0)
counts            

****Regrouping to plot the bats with a legend showing the individual species over the period****

In [ ]:
count = battiest.pivot(index='years', columns='commonName').fillna(0)
count.reset_index()
count.columns

count.plot(kind='line', figsize=(16,12))
plt.title('Bat Population of the UK over time, by decade', fontsize=20, color='Blue')
plt.xlabel('Years by decade')
plt.ylabel('Total Count')


****Beginning building a map for the overview of the bats in the UK, this is shown in the report for TMA02****

In [ ]:
# The create_map() saves an HTML representation of the map object to the specified file.
# (You can open this file in your browser as well as it displaying directly in the Notebook.)
# This will not display a map if your are offline. A network connection is required to 
# retrieve the map tiles.

# Once you've run this cell, and the map is rendered below, you'll see, 
# if you put your mouse over it and scroll, that it is zoomable.

map_TMA = folium.Map(location=[55.3781,3.4360], width=960, height=500, zoom_start=5, max_zoom=16)

map_TMA.create_map(path='data/TMA.html')

#  Due to a bug in the current version of folium, the following line is needed to force
#  the named map to be displayed in the output cell. 
map_TMA.render_iframe = True

# This is the standard Notebook way of displaying the last object named in the cell.
# It's a map object.
map_TMA

****Bringing in latitude and longitude as described below****

In [ ]:
#Reading in the first five rows of converted latitude and longitude data using the head() command.
#I converted the latitude and longitude in the grid-refs-to-lat-lon notebook and added them to a csv 
#file data/latlong.csv(manually)
coord_df = pd.read_csv('data/latlong.csv')
coord_df.head()

In [ ]:
df2 = pd.read_csv('data/greybats.csv')
df2

In [ ]:
#add markers, one for bats in each area 
# creating a marker for each.
for i in range(df2.shape[0]):
    (lat, long) = df2.ix[i,['latitude','longitude']]
    map_TMA.polygon_marker(location=[lat, long], radius=5,
                              popup='Area with bats.', 
                              line_color='blue', fill_color='#132b5e', 
                               fill_opacity=0.5)

map_TMA.render_iframe = True
# This will not display a map if your are offline. 
map_TMA.create_map(path='data/greybats.html')
map_TMA

In [ ]:
coord_df.columns

****Adding the bat name so I can put them on a pop-up on the map****

In [ ]:
coord_df['name']= hib_df['commonName']
coord_df.columns

In [ ]:
# Looping to add map data:
for i in range(coord_df.shape[0]):
    (lat, long) = coord_df.ix[i,['Latitude','Longitude']]
    print(lat, long, 'Area with bats.')

****Initial testing****

In [ ]:
#add markers, one for bats in each area 
# creating a marker for each.
for i in range(coord_df.shape[0]):
    (lat, long) = coord_df.ix[i,['Latitude','Longitude']]
    map_TMA.polygon_marker(location=[lat, long], radius=5,
                              popup='Area with bats.', 
                              line_color='blue', fill_color='#132b5e', 
                               fill_opacity=0.5)

map_TMA.render_iframe = True
# This will not display a map if your are offline. 
map_TMA.create_map(path='data/bats.html')
map_TMA

****The map of bats in the UK with pop-ups of the names. For the EMA this will be improved****

In [ ]:
# let's use the pop-up string on simple_markers to report number of bats.
#add markers, one for bats in each area 
# creating a marker for each.
for i in range(coord_df.shape[0]):
    Bats = coord_df['name'][i]
    Popstring = ('Bat Type ' + Bats)
    (lat, long) = coord_df.ix[i,['Latitude','Longitude']]
    map_TMA.simple_marker(location=[lat, long], popup=Popstring)
map_TMA.render_iframe = True
# This will not display a map if your are offline. 
map_TMA.create_map(path='data/bats.html')
map_TMA


****This section is for the EMA mainly, but is still useful for continuing this investigation. It is not complete yet****

In [ ]:
import matplotlib.patches as mpatches
# We're going to build the legend patch list one element at a time;
# it starts empty.
legendpatch_list = []
colours = {'Soprano Pipistrelle':'red', 'Common Pipistrelle':'red', "Brandt's Bat":'red',
       'Western Barbastelle':'red', 'Pipistrelle':'red', 'Brown Long-eared Bat':'red',
       "Daubenton's Bat":'red', 'Bat':'red', 'Serotine':'red', 'Greater Horseshoe Bat':'red',
       'Grey Long-eared Bat':'red', 'Long-eared Bat species':'red',
       'Lesser Horseshoe Bat':'red', "Bechstein's Bat":'red', "Whiskered/Brandt's Bat":'red',
       "Natterer's Bat":'red', 'Mouse-eared Bat':'red', 'Lesser Noctule':'red',
       'Whiskered Bat':'red', 'Myotis Bat species':'red', 'Noctule Bat':'red'}
# We want to loop for each Region
grouped = newer_df.groupby('commonName')
for key, group in grouped:
    
#for key, group in newer_df:
    # for each type create the legend patch
    legendpatch_list = legendpatch_list+([mpatches.Patch(color=colours[key], label=key)]) 

group.plot.scatter(ax=ax,x='years', y='Count', s=(group['Count']), c=colours[key])
#can't use common name on y because it isn't numeric you idiot

In [ ]:
import matplotlib.patches as mpatches
# We're going to build the legend patch list one element at a time;
# it starts empty.
legendpatch_list = []
colours = {'Soprano Pipistrelle':'red', 'Common Pipistrelle':'red', "Brandt's Bat":'red',
       'Western Barbastelle':'red', 'Pipistrelle':'red', 'Brown Long-eared Bat':'red',
       "Daubenton's Bat":'red', 'Bat':'red', 'Serotine':'red', 'Greater Horseshoe Bat':'red',
       'Grey Long-eared Bat':'red', 'Long-eared Bat species':'red',
       'Lesser Horseshoe Bat':'red', "Bechstein's Bat":'red', "Whiskered/Brandt's Bat":'red',
       "Natterer's Bat":'red', 'Mouse-eared Bat':'red', 'Lesser Noctule':'red',
       'Whiskered Bat':'red', 'Myotis Bat species':'red', 'Noctule Bat':'red'}
# We want to loop for each Region
#grouped = tidyfull_df.groupby('Region')
#for key, group in grouped:
for key in colours:
    # for each type create the legend patch
    legendpatch_list = legendpatch_list+([mpatches.Patch(color=colours[key], label=key)]) 
legendpatch_list
plt.scatter(newer_df.years, newer_df.Count, s=newer_df.Count, c=colours[key])
#can't use common name on y because it isn't numeric you idiot

In [ ]:
newer_df = new_df[['commonName','years','Count']]

In [ ]:
newer_df.columns

In [ ]:
hmmm = battier.reset_index()
hmmm.columns

In [ ]:
hmmm.plot.scatter(x='years', y='Count', c='Count')

In [ ]:
grouped = newer_df.groupby(['commonName', 'years'])
grouped.reset_index()
grouped .plot.scatter(x='years', y='Count', logx=True,
                         s=(grouped['Count']))

In [ ]:
# plt.subplots() creates a plotting object that can accept 
# multiple plots, this is embedded in a single figure.
fig, ax = plt.subplots()

#This should work, don't cnahge

# Set up the colour dict for each region:

colours = {'Soprano Pipistrelle':'red', 'Common Pipistrelle':'red', "Brandt's Bat":'red',
       'Western Barbastelle':'red', 'Pipistrelle':'red', 'Brown Long-eared Bat':'red',
       "Daubenton's Bat":'red', 'Bat':'red', 'Serotine':'red', 'Greater Horseshoe Bat':'red',
       'Grey Long-eared Bat':'red', 'Long-eared Bat species':'red',
       'Lesser Horseshoe Bat':'red', "Bechstein's Bat":'red', "Whiskered/Brandt's Bat":'red',
       "Natterer's Bat":'red', 'Mouse-eared Bat':'red', 'Lesser Noctule':'red',
       'Whiskered Bat':'red', 'Myotis Bat species':'red', 'Noctule Bat':'red'}

# We're going to be responsibile for our own legend, so we need 
# mpatches again.
import matplotlib.patches as mpatches
# We're going to build the legend patch list one element at a time;
# it starts empty.
legendpatch_list = []

# We want to loop for each Region
#grouped = tidyfull_df.groupby('Region')
#for key, group in grouped:
for key, group in grouped:
    # for each typecreate the legend patch
    legendpatch_list = legendpatch_list+([mpatches.Patch(color=colours[key], label=key)]) 
    # and plot the data, explicitly telling plot() that the 
    # axes object to plot on is the same axes object created 
    # using the .subplot() so, ax=ax is used each time.
    grouped.plot.scatter(ax=ax,x='years', y='Count',
                       logx=True,
                       s=(group['commonName']),
                        c=colours[key],
                        ylim=[35,90],
                        figsize=(8,5))
    
#plt.xticks(['1960','1970','1980','1990','2000','2010','2020'])             

# The original x and y axes labels:
plt.xlabel('Years by decade')
plt.ylabel('Total Count')

plt.savefig('data/batscrazy.png', dpi=200)

****Weather data exploration to be used for EMA****

In [ ]:
#Reading in the weather data over the period and concatenating it.
london_df = pd.read_csv('data/London Weatherdata/LondonWeather2005.csv', usecols = ['GMT', 'Mean TemperatureC', 'Precipitationmm'])
london_df1 = pd.read_csv('data/LondonWeatherdata/LondonWeather2010.csv', usecols = ['GMT', 'Mean TemperatureC', 'Precipitationmm'])
london_df2 = pd.read_csv('data/LondonWeatherdata/LondonWeather2015.csv', usecols = ['GMT', 'Mean TemperatureC', 'Precipitationmm'])

In [ ]:
#Concatenating
frames = [london_df, london_df1, london_df2]
result = pd.concat(frames).fillna(0)
result['GMT']= pd.to_datetime(result['GMT']).dt.year
result.head()

In [ ]:
#Grouping to plot
result_grouped = result.groupby(['GMT'])['Mean TemperatureC','Precipitationmm'].mean()
result_grouped.plot.bar(subplots=True, figsize=(10,10))

In [ ]:
#Reading in the weather data over the period and concatenating it.
leeds_df = pd.read_csv('data/LeedsWeatherdata/LeedsWeather2001.csv', usecols = ['GMT', 'Mean TemperatureC', 'Precipitationmm'])
leeds_df1 = pd.read_csv('data/LeedsWeatherdata/LeedsWeather2009.csv', usecols = ['GMT', 'Mean TemperatureC', 'Precipitationmm'])
leeds_df2 = pd.read_csv('data/LeedsWeatherdata/LeedsWeather2015.csv', usecols = ['GMT', 'Mean TemperatureC', 'Precipitationmm'])

In [ ]:
#Concatenating
frames1 = [leeds_df, leeds_df1, leeds_df2]
result1 = pd.concat(frames1).fillna(0)
result1['GMT']= pd.to_datetime(result1['GMT']).dt.year
result1.head()

In [ ]:
#Grouping to plot
result1_grouped = result1.groupby(['GMT'])['Mean TemperatureC','Precipitationmm'].mean()

In [ ]:
#plotting Leeds weather fropm 1996 - 2015
result1_grouped.plot.bar(subplots=True, figsize=(10,10), color='white')

In [ ]:
#Reading in the weather data over the period and concatenating it.
Manch_df = pd.read_csv('data/ManchesterWeatherdata/ManchesterWeather2001.csv', 
                       usecols = ['GMT', 'Mean TemperatureC', 'Precipitationmm'])
Manch_df1 = pd.read_csv('data/ManchesterWeatherdata/ManchesterWeather2009.csv', 
                        usecols = ['GMT', 'Mean TemperatureC', 'Precipitationmm'])
Manch_df2 = pd.read_csv('data/ManchesterWeatherdata/ManchesterWeather2015.csv', 
                        usecols = ['GMT', 'Mean TemperatureC', 'Precipitationmm'])

In [ ]:
#Concatenating
frames2 = [Manch_df, Manch_df1, Manch_df2]
result2 = pd.concat(frames2).fillna(0)
result2['GMT']= pd.to_datetime(result2['GMT']).dt.year
result2.head()

In [ ]:
#Grouping to plot
result2_grouped = result2.groupby(['GMT'])['Mean TemperatureC','Precipitationmm'].mean()

In [ ]:
#plotting Leeds weather fropm 1996 - 2015
result2_grouped.plot.bar(subplots=True, figsize=(10,10))

In [ ]:
ax = result_grouped.plot.bar()
result1_grouped.plot.bar(ax=ax, figsize=(15,10), color='white')
plt.title('London and Leeds weather over time from 2001 to 2015', fontsize=20, color='Blue')

Having studied the weather data over the period it can be seen that there is a peak in temperature in 2005 for both Leeds and
London, although the weather mean temperature in London is higher overall. It can also be seen that at the same time as the
temperature increased so did the precipitation. It appears that bats must like warm temperatures though because the 
overall bat population increased in line with the temperature increases, however they may not be big fans of rain because the 
populations are decreasing in line with the increases in precipitation. I attempted to plot Cardiff, Manchester and Bristol data, 
but apparently it didn't rain over the period which I think is highly unlikely. However they followed the same temperature trends 
as London and Leeds. There was also the issue of no records for weather for earlier years which made the results a guide only.
